In [1]:
import matplotlib.pyplot as plt
import numpy as nm
import pandas as pd
import scipy as sp
import PIL
import cv2
import math

In [2]:
from PIL import Image

img1 = Image.open('WaterMark.bmp')
bit =nm.asarray(img1)




img = Image.open('cat.bmp').convert('L')



In [3]:
"""def svd(A):
    ATA = A.T @ A

    iden = nm.eye(2)

    err = 1e-5

    for i in range(100):
        Q , R = nm.linalg.qr(ATA)
        iden = iden @ Q
        ATA = R @ Q
        if(nm.sum( err > nm.abs(ATA - nm.diag(nm.diagonal(ATA))))):
            break
        
    
    D = nm.sqrt(nm.diagonal(ATA))
    sira = nm.argsort(-D)
    iden = iden[:, sira]
    V = iden
    D = D[sira]

    U = A @ V / D

    return U,D,V.T"""




'def svd(A):\n    ATA = A.T @ A\n\n    iden = nm.eye(2)\n\n    err = 1e-5\n\n    for i in range(100):\n        Q , R = nm.linalg.qr(ATA)\n        iden = iden @ Q\n        ATA = R @ Q\n        if(nm.sum( err > nm.abs(ATA - nm.diag(nm.diagonal(ATA))))):\n            break\n        \n    \n    D = nm.sqrt(nm.diagonal(ATA))\n    sira = nm.argsort(-D)\n    iden = iden[:, sira]\n    V = iden\n    D = D[sira]\n\n    U = A @ V / D\n\n    return U,D,V.T'

In [5]:
def attack(img_attack):
    data_1 = nm.array(img_attack)
    data = nm.array(data_1,dtype=nm.int64)



    Q=7



    block_size=2
    height, width = data.shape
    blocks = []
    s=0
    for y in range(0, height, block_size): #bakıcaz
        for x in range(0, width, block_size):
            block = data[y:y + block_size, x:x + block_size]
            blocks.append(block)


    for indx in range(len(blocks)):

        block = nm.array(blocks[indx],dtype=nm.int64)

        U, eigenvalues, Vt = nm.linalg.svd(block)

        Z =  eigenvalues[0] % Q

        #print(eigenvalues)



        
        if (bit[((s*block_size) // width)][(s)%(width // block_size)] == 0):
            #print(0)
            if (Z < (3* (Q / 4))):
                
                eigenvalues[0] = eigenvalues[0] + (Q/4) - Z
            else:
                eigenvalues[0] = eigenvalues[0] + (5*Q /4) -Z

        else:
            #print(1)
            if ( Z < (Q / 4)):
                eigenvalues[0] = eigenvalues[0] -(Q / 4) + Z
            else:
                eigenvalues[0] = eigenvalues[0] +(3*Q / 4) -Z
        


        blocks[indx] = U @ nm.diag(eigenvalues) @ Vt



        s += 1

        #ReConstruct img
    #print(blocks[0][0])


    reconstructed_image = nm.zeros((height, width), dtype=nm.float64)

    block_idx = 0  # Start from the first block in the list
    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            # Get the block and place it in the appropriate position
            block = blocks[block_idx]
            reconstructed_image[y:y + block_size, x:x + block_size] = block
            block_idx += 1  # Move to the next block

    #print(data[0][0])
    #print(reconstructed_image[0][0])

    reconstructed_pil_image = Image.fromarray((reconstructed_image).astype(nm.uint8))
    

    reconstructed_pil_image.save('attack.bmp')
    return reconstructed_pil_image

In [6]:

def extract(img_to_extract):

    
    
    data_1 = nm.array(img_to_extract)
    data = nm.array(data_1,dtype=nm.int64)



    Q=7


    block_size=2
    height, width = data.shape
    blocks = []
    s=0
    for y in range(0, height, block_size): #bakıcaz
        for x in range(0, width, block_size):
            block = data[y:y + block_size, x:x + block_size]
            blocks.append(block)
    
    extract_watermark = nm.zeros((256*256), dtype=nm.bool_)
    reshaped =extract_watermark.reshape((256,256))
    for block in blocks:
        #SVD allaha emanet
        block = nm.array(block,dtype=nm.int64)

        U, eigenvalues, Vt = nm.linalg.svd(block)

        Z =  eigenvalues[0] % Q

        
        if (Z < Q//2):
            extract_watermark[s] = False
        else:
            extract_watermark[s] = True

    
        s += 1



    reshaped =extract_watermark.reshape((256,256))



    pixel_array = reshaped * 255
    EXT= Image.fromarray(pixel_array.astype(nm.uint8))

    EXT.save('lastWM.bmp')
    return EXT

            


In [7]:
attacked = attack(img)

In [8]:

water_mark = extract(attacked)

In [9]:
def mean_squared_error(image1, image2):
    # Ensure both images have the same dimensions
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same dimensions")
    
    # Calculate MSE
    return nm.mean((image1.astype(nm.float32) - image2.astype(nm.float32)) ** 2)

# Function to calculate PSNR
def calculate_psnr(image1, image2):
    mse = mean_squared_error(image1, image2)
    
    if mse == 0:
        # If MSE is zero, the images are identical
        return float("inf")
    
    # Maximum pixel value for 8-bit images
    max_pixel = 255.0
    return 10 * math.log10((max_pixel ** 2) / mse)

# Load two grayscale images to compare
image1 = nm.array(Image.open("cat.bmp").convert("L"))
image2 = nm.array(Image.open("cat_attacked.bmp").convert("L"))

# Calculate PSNR
psnr_value = calculate_psnr(image1, image2)
print("PSNR:", psnr_value)

PSNR: 49.32607921567465
